# 컨텐츠 기반 추천시스템

아이템 평점 기반 추천

In [1]:
import matplotlib
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
fontprop = fm.FontProperties(fname=fontpath, size=8) 

In [2]:
#폰트종류설정
plt.rcParams["font.family"] = 'NanumGothic'

#폰트크기설정
#plt.rcParams["font.size"] = 20

## 데이터 전처리

In [3]:
import konlpy
import urllib.request
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

from konlpy.tag import Mecab
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

/tmp/ipykernel_87791/691330863.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-white')
2023-03-25 12:19:56.464177: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-25 12:19:56.620271: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`

In [29]:
import pandas as pd
import numpy as np
data_df = pd.read_csv('../data/ulsan_rest_table_ver3.csv')
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54500 entries, 0 to 54499
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   p_id      54500 non-null  int64 
 1   place_id  54500 non-null  object
 2   u_id      54500 non-null  int64 
 3   user_id   54500 non-null  object
 4   score     54500 non-null  int64 
 5   comment   40225 non-null  object
dtypes: int64(3), object(3)
memory usage: 2.5+ MB


In [30]:
# data_df['comment'] = data_df['comment'].str.replace("[^ㄱ-하-ㅣ가-힣 ]", "")
# data_df['comment'].replace('', np.nan, inplace=True)
print(len(data_df))
print(data_df.isnull().sum())

54500
p_id            0
place_id        0
u_id            0
user_id         0
score           0
comment     14275
dtype: int64


In [31]:
# print(data_df.isnull().sum())

data_df = data_df.dropna(how='any')

In [32]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40225 entries, 0 to 54499
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   p_id      40225 non-null  int64 
 1   place_id  40225 non-null  object
 2   u_id      40225 non-null  int64 
 3   user_id   40225 non-null  object
 4   score     40225 non-null  int64 
 5   comment   40225 non-null  object
dtypes: int64(3), object(3)
memory usage: 2.1+ MB


In [33]:
# data_df['comment'].replace('', np.nan, inplace=True)
print(len(data_df))
print(data_df.isnull().sum())

40225
p_id        0
place_id    0
u_id        0
user_id     0
score       0
comment     0
dtype: int64


## Item Matrix

In [36]:
n_users = data_df.u_id.max()
n_places = data_df.p_id.max()
shape = (n_places+1, n_users+1)
print(shape)

(492, 24545)


In [37]:
item_Matrix = np.ndarray(shape, dtype=float)
# item_Matrix
for p_id, u_id, rating in zip(data_df.p_id, data_df.u_id, data_df.score):
    item_Matrix[p_id][u_id] = rating
item_Matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Cosine Similarity

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
similarity = cosine_similarity(item_Matrix, item_Matrix)
print('코사인 유사도 연산 결과 :',similarity.shape)

코사인 유사도 연산 결과 : (492, 492)


In [50]:
def get_recommendations(p_id, sim=similarity):
    # 해당 아이템과 모든 아이템의 유사도를 가져온다.
    sim_scores = list(enumerate(sim[p_id]))

    # 유사도에 따라 아이템들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 아이템을 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 아이템의 ID을 리턴한다.
    return sim_scores

In [51]:
get_recommendations(400)

[(338, 0.09024862773069361),
 (225, 0.05716550012797329),
 (103, 0.05473585873143489),
 (39, 0.0540662415097443),
 (35, 0.049687234845050735),
 (218, 0.04616980344706986),
 (435, 0.04549720439464462),
 (375, 0.045305856069212946),
 (410, 0.04339674475652684),
 (175, 0.03999672360259821)]

In [55]:
data_df[data_df.p_id == 400]

,p_id,place_id,u_id,user_id,score,comment
43961,400,카몬시카페,24348,황인자,5,바닷가뷰가 너무좋아요~가래떡구이 맛있어요
43963,400,카몬시카페,24278,황병모,4,정말 마음이 탁 트였고 참 좋왓습니다
43964,400,카몬시카페,24239,황경희,5,바로 코앞에서 바닷물이 춤추고 봄바람이 살랑이는 멋진곳입니다\n크레마풍성한 아메리...
43965,400,카몬시카페,24058,혼밥러,5,"뷰, 위치, 공간 다 좋음"
43966,400,카몬시카페,24049,호잇호잇,4,뷰도 좋고 커피도 맛있었어요 넓은데 사람들도 많이 오더라구요!
...,...,...,...,...,...,...
44165,400,카몬시카페,1227,DEW S,5,풍경도 좋은데 음료도 디저트도 맛있었어요. 다시 오고 싶음🥰…
44167,400,카몬시카페,987,circle,4,피자에 도우가 페스츄리빵이라 좀 별로였음
44168,400,카몬시카페,954,Chris L,5,Was great
44170,400,카몬시카페,554,binsp p,5,커피 맛나고 의자 편해서 자주 가용 하지만 허니브레드는 노맛


In [62]:
data_df[data_df.u_id == 987]

,p_id,place_id,u_id,user_id,score,comment
4255,34,곤지곤지 호계점,987,circle,4,주차가 편하고 가성비가 괜찮은편.
5091,39,곽암 아트 카페 갤러리,987,circle,4,특색은 있지만 자주 방문하게 만들것같은\n뭔가는 없는곳
11075,77,대왕암아구찜,987,circle,4,해물찜은 별로인데 아구는 괜찮음
28597,247,스타벅스 울산간절곶점,987,circle,3,항상 사람이 붐비고 바깥에 흙길을 걷다 들어오는 사람들이 많아 먼지가 많은 느낌입니다
43949,399,칠성양곱창,987,circle,4,반찬도 그렇고 곱창구이도 그렇고 음식이 좀 짜요. 응대는 친절함.
44167,400,카몬시카페,987,circle,4,피자에 도우가 페스츄리빵이라 좀 별로였음
46110,416,카페코이,987,circle,5,감성이 있는 조명이 멋진곳\n낮 보다는 해질녘 이후를 추천
49033,435,투썸플레이스 울산정자점,987,circle,5,리모델링 이후 뷰가 끝내주게 변했습니다. 바다뷰로는 전국 어느까페에도 뒤지지 않을정도
49034,435,투썸플레이스 울산정자점,987,circle,5,리모델링 이후 뷰가 끝내주게 변했습니다. 바다뷰로는 전국 어느까페에도 뒤지지 않을정도
52746,476,헤이메르,987,circle,3,빵이나 커피는 그닥 특별하진 않았고 경치가 좋음. 사람많고 높은곳에 있어서 초보운전...
